## Load in data

In [1]:
%load_ext autoreload
%autoreload 2
from ais_dataloader import *
from gp_kernel_ship_classification_dataset import *
from gp_kernel_ship_classification_trainer import *
from gp_kernel_ship_classification_network import *

import ipywidgets as widgets
from IPython.display import display
from plotting_utils import *



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
date_range = pd.date_range(start='2024-01-01', end='2024-01-01', freq='D')
gp_regression_dataset = AISTrajectoryRegressionDataset(date_range, device)


Loading cached dataframe from data/processed/processed_AIS_df_2024_01_01_2024_01_01.pkl


Scaling trajectories for each MMSI: 100%|██████████| 3453/3453 [00:01<00:00, 2072.58it/s]


===== Dataset Statistics =====
Total number of AIS messages: 2128288
Number of unique MMSIs: 3453
Date range: 2024-01-01 00:00:00 to 2024-01-01 23:59:59


## Fit GP Models

In [3]:
import torch
import gpytorch
from multioutput_gp import *

num_trajectories = 3453
models = {}
likelihoods = {}
losses = {}

for idx in range(num_trajectories):
    # mmsi, times, state_trajectory = gp_regression_dataset[idx]
    mmsi, times, state_trajectory = gp_regression_dataset[idx]
    
    print(f"\nFitting GP for trajectory {idx+1}/{num_trajectories} for MMSI {mmsi}")
    
    X = times.detach().unsqueeze(1).to(device)
    Y = state_trajectory.detach().to(device)
    
    num_outputs = Y.shape[1]
    
    likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=num_outputs, noise_prior=gpytorch.priors.NormalPrior(loc=0.25, scale=0.25)).to(device)
    model = MultiOutputExactGPModel(X, Y, likelihood, num_outputs=num_outputs).to(device)

    with gpytorch.settings.cholesky_jitter(1e-3):
        loss, model, likelihood = train_model(model, likelihood, X, Y, num_epochs=100, lr=0.1, mmsi=mmsi)

    print(f"Loss: {loss.item()}")
    # models.append(model)
    # likelihoods.append(likelihood)
    # losses.append(loss.item())
    models[mmsi] = model
    likelihoods[mmsi] = likelihood
    losses[mmsi] = loss.item()


Fitting GP for trajectory 1/3453 for MMSI 3660489


GP Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]/home/daniel/miniconda3/envs/gp/lib/python3.13/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-03 to the diagonal
  warnings.warn(
GP Training Progress: 100%|██████████| 100/100 [00:00<00:00, 107.08it/s]


Loss: 1.1245033740997314

Fitting GP for trajectory 2/3453 for MMSI 203661016


GP Training Progress: 100%|██████████| 100/100 [00:04<00:00, 22.26it/s]


Loss: 1.0691908597946167

Fitting GP for trajectory 3/3453 for MMSI 205691000


...

Loss: 0.7724918723106384

Fitting GP for trajectory 3452/3453 for MMSI 750000076


GP Training Progress: 100%|██████████| 100/100 [00:00<00:00, 151.67it/s]


Loss: 1.351094365119934

Fitting GP for trajectory 3453/3453 for MMSI 856583816


GP Training Progress: 100%|██████████| 100/100 [00:05<00:00, 19.73it/s]

Loss: 0.882996141910553


### Load Models if already fit

In [5]:
bundle = torch.load("kernel_classification_bundle.pt", weights_only=False)
kernel_classification_dataset = bundle["kernel_classification_dataset"]
models_state_dict = bundle["models_state_dict"]
likelihoods_state_dict = bundle["likelihoods_state_dict"]
losses = bundle["losses"]

In [7]:
print(dir(gp_regression_dataset))

['KNOTS_TO_METERS_PER_SECOND', '__add__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_init_vessel_group_mappings', 'ais_to_state_space', 'create_combined_df', 'device', 'df', 'get_vessel_group_by_mmsi', 'get_vessel_group_id_by_mmsi', 'map_to_vessel_group', 'print_stats', 'process_AIS_data', 'remove_AIS_artifacts', 'scale_trajectories', 'scaled_trajectories_by_mmsi', 'scalers_by_mmsi', 'trajectories_by_mmsi', 'vessel_group_id_to_group', 'vessel_group_to_id']


### Assess a single model kernel

In [4]:
model = next(iter(models.values()))  # Get one of the models for demonstration
for param_name, param in model.named_parameters():
    print(f'Parameter name: {param_name:42} value = {param.tolist()}')
    
print()
print(model.covar_module.data_covar_module.kernels[0].lengthscale.item())

Parameter name: likelihood.raw_task_noises                 value = [-0.690277099609375, -0.5817338228225708, -0.9445188641548157, -0.5356355905532837, -0.9430439472198486, -0.9433724880218506]
Parameter name: likelihood.raw_noise                       value = [-0.9224146604537964]
Parameter name: mean_module.base_means.0.raw_constant      value = 0.181679829955101
Parameter name: mean_module.base_means.1.raw_constant      value = -0.04717198759317398
Parameter name: mean_module.base_means.2.raw_constant      value = -0.00033445091685280204
Parameter name: mean_module.base_means.3.raw_constant      value = -0.0047524720430374146
Parameter name: mean_module.base_means.4.raw_constant      value = -2.4937369744293392e-05
Parameter name: mean_module.base_means.5.raw_constant      value = 8.134491054079263e-07
Parameter name: covar_module.task_covar_module.covar_factor value = [[-0.43839308619499207], [-0.06146915256977081], [0.011572202667593956], [-0.03653256967663765], [0.0008144288440234

In [5]:
# Set the model and likelihood to evaluation mode
model.eval()
likelihood.eval()

# Generate test inputs (e.g., evenly spaced time points)
test_times = torch.linspace(times.min(), times.max(), 200).unsqueeze(1).to(device)

# Make predictions
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.cholesky_jitter(1e-3):
    predictions = likelihood(model(test_times))
    mean = predictions.mean
    lower, upper = predictions.confidence_region()



### Plot GP Solution

In [6]:
# model.eval()
# likelihood.eval()
# from plotting_utils import *

# for mmsi in models:
#     model = models[mmsi]
#     likelihood = likelihoods[mmsi]
#     # Get the corresponding data for this MMSI
#     # If you want to use the same train/test split as before:
#     times, state_trajectory = None, None
#     for entry in gp_regression_dataset:
#         if entry[0] == mmsi:
#             _, times, state_trajectory = entry
#             break
#     if times is None:
#         continue  # skip if MMSI not found

#     train_X = times.clone().detach().unsqueeze(1).cpu()
#     train_Y = state_trajectory.clone().detach().cpu()

#     test_X = torch.linspace(times.min(), times.max(), 500).unsqueeze(1).to(device)
    
#     test_Y = eval_model(model, likelihood, test_X)

#     plot_gp(train_X, train_Y, test_X, test_Y)
#     plot_single_ship_path(mmsi, times, state_trajectory)


In [7]:
# import ipywidgets as widgets
# from IPython.display import display, clear_output
# from plotting_utils import *


# def plot_for_mmsi(selected_mmsi):
#     clear_output(wait=True)
#     model = models[selected_mmsi]
#     likelihood = likelihoods[selected_mmsi]
#     # Get the corresponding data for this MMSI
#     times, state_trajectory = None, None
#     for entry in gp_regression_dataset:
#         if entry[0] == selected_mmsi:
#             _, times, state_trajectory = entry
#             break
#     if times is None:
#         print("No data for MMSI:", selected_mmsi)
#         return

#     train_X = times.clone().detach().unsqueeze(1).cpu()
#     train_Y = state_trajectory.clone().detach().cpu()
#     test_X = torch.linspace(times.min(), times.max(), 500).unsqueeze(1).to(device)
#     test_Y = eval_model(model, likelihood, test_X)

#     plot_gp(train_X, train_Y, test_X, test_Y)
#     plot_single_ship_path(selected_mmsi, times, state_trajectory)

    
# mmsi_dropdown = widgets.Dropdown(
#     options=list(models.keys()),
#     description='MMSI:',
#     disabled=False,
# )

# widgets.interact(plot_for_mmsi, selected_mmsi=mmsi_dropdown)
    
    
    

In [8]:
print(pd.unique(gp_regression_dataset.df['MMSI'].values))
gp_regression_dataset.get_vessel_group_by_mmsi(3660489)

[367669550 367118980 636018568 ... 367619000 309108000 368926390]


'Other'

## Create the kernel param to ship mmsi dataset


In [ ]:
kernel_classification_dataset = GPKernelShipClassificationDataset(gp_regression_dataset, models, device, scalers_by_mmsi=gp_regression_dataset.scalers_by_mmsi)
unique_group_ids = kernel_classification_dataset.get_unique_group_ids()

train_classification_dataset, test_classification_dataset = torch.utils.data.random_split(kernel_classification_dataset, [0.8, 0.2])
train_loader = torch.utils.data.DataLoader(train_classification_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_classification_dataset, batch_size=32, shuffle=False)

In [13]:
# torch.save(kernel_classification_dataset, 'kernel_classification_dataset.pt')
save_dict = {
    "kernel_classification_dataset": kernel_classification_dataset,  # Only if this is safe to pickle!
    "models_state_dict": {k: v.state_dict() for k, v in models.items()},
    "likelihoods_state_dict": {k: v.state_dict() for k, v in likelihoods.items()},
    "losses": losses,
}
torch.save(save_dict, "kernel_classification_bundle.pt")

## Train the classification model

In [16]:
from gp_kernel_ship_classification_network import GPKernelShipClassificationNetwork
model = GPKernelShipClassificationNetwork(input_dim=2, num_classes=max(unique_group_ids) + 1).to(device)  # +1 for background class

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

trainer = GPKernelShipClassificationTrainer(model, train_loader, test_loader, criterion, optimizer, device)
trainer.train(num_epochs=500)

Epoch 500/500 | Train Loss: 1.6009 | Train Acc: 0.3768 | Test Loss: 1.7381 | Test Acc: 0.3333
